In [ ]:
#Import the library
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import os

In [ ]:
# NRCS mehtod using least-squares
def quickflow_scs(S, P, a):
    """
    Calculate runoff using the NRCS method.
    Parameters:
    - S: Soil retention parameter
    - P: Total precipitation (array)
    - a: Initial abstraction ratio

    Returns:
    - Array of quickflow values, computed using the NRCS-CN method formula
    """
    Q = ((P - a * S) ** 2) / (P + (1 - a) * S)
    Q[P <= a * S] = 0  # No runoff if precipitation is less than the threshold
    return Q

# Objective function for the optimization problem
def objetivo(S, P, Qo, a):
    Q_estimado = quickflow_scs(S, P, a)
    return np.sum((Qo - Q_estimado) ** 2)

def optimize_CN(file):
    df = pd.read_csv(file)
    Qo = df['Quickflow_eckhardt'].values
    P = df['total_precipitation_sum'].values
    results = {'file': file}
    S_min = (25400 / 99.90) - 254  # Minimum S value, corresponding to CN = 100
    S_max = (25400 / 0.1) - 254  # Maximum S value, corresponding to CN near zero
    
# Loop over different values of initial abstraction ratio    
    for a in [0.05,0.2]:
            best_result = None
            best_CN = None

            for initial_guess in np.linspace(0, 3000, 25):
                result = minimize(objetivo, initial_guess, args=(P, Qo, a), bounds=[(S_min, S_max)])
                
                if (best_result is None) or (result.fun < best_result.fun):
                    best_result = result
                    S_optimized = result.x[0]
                    best_CN = 25400 / (S_optimized + 254)

            Q_estimated = quickflow_scs(((25400 /best_CN) - 254), P, a)
            mse, rmse, mae = error_metrics(Qo, Q_estimated)
            pbias = PBIAS(Qo, Q_estimated)
            nse = NSE(Qo, Q_estimated)
            kge = KGE(Qo, Q_estimated)
            r2_score = r2(Qo, Q_estimated)
            rse = RSE(Qo, Q_estimated)
            
            results[f'CN_Optimized_a_{a}'] = best_CN         
            results[f'R2_a_{a}'] = r2_score
            results[f'PBIAS_a_{a}'] = pbias
            results[f'NSE_a_{a}'] = nse
            results[f'KGE_a_{a}'] = kge
            results[f'RMSE_a_{a}'] = rmse
            df[f'Q_estimated_a_{a}'] = Q_estimated
       
    return results,df
# Statistical error metrics
def error_metrics(observed, estimated):
    residuals = observed - estimated
    mse = np.mean(residuals**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(residuals))
    return mse, rmse, mae

def PBIAS(observed, simulated):
    return (np.sum(observed - simulated) / np.sum(observed)) * 100

def NSE(observed, simulated):
    return 1 - (np.sum((observed - simulated) ** 2) / np.sum((observed - np.mean(observed)) ** 2))

def KGE(observed, simulated):
    r = np.corrcoef(observed, simulated)[0, 1]
    alpha = np.std(simulated) / np.std(observed)
    beta = np.mean(simulated) / np.mean(observed)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)

def r2(y_true, y_pred):
    correlation_matrix = np.corrcoef(y_true, y_pred)
    correlation_xy = correlation_matrix[0,1]
    return correlation_xy**2

def RSE(observed, simulated):
    observed = np.array(observed)
    simulated = np.array(simulated)
    n = len(observed)
    
    if n <= 2:
        return np.nan  # Return NaN if we can't compute RSE
    
    # Mean of the observed data
    y_obs_mean = np.mean(observed)
    sum_squared_diff = np.sum((observed - simulated) ** 2)
    sum_squared_total = np.sum((observed - y_obs_mean) ** 2)
    
    # If sum_squared_total is 0, RSE cannot be calculated as it would involve division by zero
    if sum_squared_total == 0:
        return np.nan
    return np.sqrt((n * sum_squared_diff) / ((n - 2) * sum_squared_total))

# Main function to process files within a folder and compile results into a CSV file
folder_path = r'\Times_series\LS'  # Specify the folder containing the series
csv_files = [os.path.join(root, name) for root, dirs, files in os.walk(folder_path) for name in files if name.endswith(".csv")]

results_list = []

for file in csv_files:
    results, df_modified = optimize_CN(file)
    file_name = os.path.basename(file)
    results['file'] = file_name  
    df_modified.to_csv(file, index=False)
    results_list.append(results)

df_results = pd.DataFrame(results_list)
df_results.to_csv(os.path.join(folder_path,'results_LS_G1.csv'), index=False) 
print(df_results)
file_path = r"\Times_series\LS"  